In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [10]:
# 데이터 로드

data_path = r'C:\Users\csh16\Desktop\2024-2\졸업프로젝트\dataset\cluster_with_labels_modified.csv'
df = pd.read_csv(data_path, encoding='utf-8')

# '이용금액_업종기준', '이용금액_요식', '이용금액_교육', '여유_Pet이용금액', '이용금액_여행', '이용금액_자차',
# '이용금액_대중교통', '이용금액_여가활동', '이용금액_일상생활_총합' 컬럼만 사용
data_columns = ['이용금액_업종기준', '이용금액_요식', '이용금액_교육', '여유_Pet이용금액', '이용금액_여행',
                '이용금액_자차', '이용금액_대중교통', '이용금액_여가활동', '이용금액_일상생활_총합']
labels_column = 'Cluster'

# 필요한 데이터만 추출
data = df[data_columns].values
labels = df[labels_column].values

# 데이터 표준화 (스케일링)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# 데이터를 torch tensor로 변환
data_tensor = torch.tensor(data_scaled, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# 배치 차원 추가 (batch_size, sequence_length, input_size)
data_tensor = data_tensor.unsqueeze(1)  # 여기서는 1 타임스텝 시퀀스로 처리

In [11]:
# GRU 모델 정의
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()  # 활성화 함수 추가
    
    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝의 출력을 사용
        return out

In [12]:
# 2. FGSM 공격을 위한 함수 정의
def fgsm_attack(data, epsilon, data_grad):
    # 데이터의 기울기를 이용하여 epsilon 범위 내에서 변동을 줌
    sign_data_grad = data_grad.sign()
    perturbed_data = data + epsilon * sign_data_grad
    return perturbed_data

# 3. 손실 함수 및 평가 모드 설정
criterion = nn.CrossEntropyLoss()
epsilon = 0.01  # 적대적 변동 크기

In [13]:
# 4. 소비 데이터와 라벨을 불러오기 (예시로 더미 데이터 사용)
# 여기서 data는 (배치 크기, 시퀀스 길이, 특성 수) 형식이어야 하며,
# labels는 (배치 크기) 형식이어야 합니다.
# 실제로는 데이터를 불러오거나 전처리하는 과정이 필요함
data = torch.randn(32, 10, 9)  # 예시: 배치 크기 32, 시퀀스 길이 10, 특성 수 9
labels = torch.randint(0, 25, (32,))  # 예시: 25개의 클래스 (랜덤한 라벨)

In [14]:
# 5. 모델 초기화
input_size = 9  # 사용한 컬럼 수
hidden_size = 64
num_layers = 1
output_size = 25  # 출력 클래스 수

# 모델 초기화 및 가중치 불러오기
model = GRUModel(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load(r'C:\Users\csh16\Desktop\2024-2\졸업프로젝트\gru_model_weights.pth'))
model.eval()  # 평가 모드로 전환

GRUModel(
  (gru): GRU(9, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=25, bias=True)
)

In [15]:
# 6. 적대적 예제 생성 및 실험
data.requires_grad = True  # 입력 데이터에 대한 기울기 계산 활성화

# 모델 예측
output = model(data)
loss = criterion(output, labels)

# 역전파
model.zero_grad()
loss.backward()

# 데이터 기울기 추출
data_grad = data.grad.data

# FGSM 공격 적용
perturbed_data = fgsm_attack(data, epsilon, data_grad)

# 적대적 예제의 예측 확인
output_perturbed = model(perturbed_data)
pred_perturbed = output_perturbed.argmax(dim=1, keepdim=True)

# 결과 출력
print("Original Prediction: ", labels)
print("Adversarial Prediction: ", pred_perturbed)

Original Prediction:  tensor([12, 20, 18,  0, 19, 16, 22,  5, 17, 10, 11, 23, 23,  7,  9, 16,  6, 16,
        15, 21,  1, 11, 20,  7, 10, 21,  8, 15,  3, 16, 23,  9])
Adversarial Prediction:  tensor([[ 9],
        [18],
        [ 0],
        [13],
        [ 0],
        [ 4],
        [ 0],
        [20],
        [ 4],
        [13],
        [ 6],
        [13],
        [ 0],
        [ 4],
        [ 0],
        [ 0],
        [ 6],
        [ 0],
        [21],
        [20],
        [ 7],
        [20],
        [13],
        [20],
        [ 4],
        [20],
        [ 0],
        [13],
        [20],
        [ 0],
        [ 0],
        [21]])
